# SST regression maps

In [ ]:
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import t2da, t2ds
from SST import SST_index, EOF_SST_analysis
from maps import map_robinson, map_eq_earth, rect_polygon, make_map, regr_map
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rect
from plotting import shifted_color_map, discrete_cmap
from timeseries import IterateOutputCESM, lowpass, chebychev
from xr_DataArrays import xr_AREA
from xr_regression import xr_lintrend, xr_linear_trends_2D, xr_linear_trend,\
                          ocn_field_regression, lag_linregress_3D
from regression_maps import SST_regr_standard, SST_regr_lpd, SST_regr_lpi

In [ ]:
from xr_DataArrays import depth_lat_lon_names

In [ ]:
from derive_files import MakeDerivedFiles

In [ ]:
lpd = MakeDerivedFiles('lpd')

In [ ]:
from paths import path_samoc

In [ ]:
from xr_regression import autocorrelation_3D

In [ ]:
SST = xr.open_dataarray(f'{path_samoc}/SST/SST_GMST_dt_yrly_had.nc', decode_times=False)

In [ ]:
SST.sel(longitude=-160, latitude=0, method='nearest').plot()

In [ ]:
def autocorrelation_3D(x):
    """ autocorrelation 
    can deal with NaNs in time dimension, e.g. in HadISST
    """    
    x, y = x[1:], x.shift(time=1)[1:]
    y = xr.where(np.isnan(x), np.nan, y)
    x = xr.where(np.isnan(y), np.nan, x)

    n     = xr.where(np.isnan(x), 0, 1).sum(axis=0)
    xmean = x.mean(axis=0, skipna=True)
    ymean = y.mean(axis=0, skipna=True)
    xstd  = x.std( axis=0, skipna=True)
    ystd  = y.std( axis=0, skipna=True)
    
    x -= xmean
    y -= ymean
    
    cov      = np.divide(np.nansum(np.multiply(x,y), axis=0), n)
    cor      = cov/(xstd*ystd)
    cor.name = 'autocorrelation'
    
    return cor

In [ ]:
%%time
ds = autocorrelation_3D(SST)


In [ ]:
ds.plot(vmin=0)

In [ ]:
np.isnan(x.shift(time=1).values[0])

In [ ]:
xr.where(np.isnan(x.shift(time=1)), 5, x)

In [ ]:
x[1:]

In [ ]:
x.shift(time=1)[1:]

In [ ]:
%%time
lpd.make_autocorrelation_map()

In [ ]:
%%time
had.make_SST_autocorrelation_map()

In [ ]:
da = xr.open_dataarray(f'{path_samoc}/SST/SST_autocorrelation_had.nc')

In [ ]:
da.plot()

In [ ]:
from timeseries import TimeSeriesSynthesis

In [ ]:
tss = TimeSeriesSynthesis()

In [ ]:
tss.ctrl['GMST'].plot()
tss.ctrl['AMO'].plot()
tss.ctrl['SOM'].plot()


In [ ]:
A['name'] = 'ss'

In [ ]:
A

In [ ]:
SST_yrly_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_GMST_dt_yrly_ctrl.nc', decode_times=False)
SST_yrly_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_GMST_dt_yrly_rcp.nc' , decode_times=False)
SST_yrly_lpd  = xr.open_dataarray(f'{path_samoc}/SST/SST_GMST_dt_yrly_lpd.nc' , decode_times=False)
SST_yrly_lpi  = xr.open_dataarray(f'{path_samoc}/SST/SST_GMST_dt_yrly_lpi.nc' , decode_times=False)
SST_yrly_had  = xr.open_dataarray(f'{path_samoc}/SST/SST_GMST_dt_yrly_had.nc' , decode_times=False)

In [ ]:
SST_yrly_lpd

In [ ]:
def autocorrelation(x):
    return xr.DataArray(np.corrcoef(x[1:], x[:-1])[0,1])

def autocorrelation_map(field):
    """ autocorrelation along axis 0 """
    ac_field = xr.apply_ufunc(autocorrelation, field, input_core_dims=[['time']])
    
    return ac_field

def xr_autocorrelation_2D(da, dim_names, with_nans=False):
    """ calculate linear trend of 2D field in time
    
    input:
    da        .. 3D xr DataArray with (dim_names) dimensions
    dim_names .. tuple of 2 strings: e.g. lat, lon dimension names
    
    output:
    da_trend  .. slope of linear regression
    """
    (dim1, dim2) = dim_names
    # stack lat and lon into a single dimension called allpoints
    stacked = da.stack(allpoints=[dim1, dim2])
    # apply the function over allpoints to calculate the trend at each point
    if with_nans==False:
        trend = stacked.groupby('allpoints').apply(autocorrelation)
        # unstack back to lat lon coordinates
        da_ac = trend.unstack('allpoints')
    if with_nans==True:
        trend = stacked.groupby('allpoints').apply(xr_linear_trend_with_nans)
        # unstack back to lat lon coordinates
        da_ac = trend.unstack('allpoints')
    da_ac = da_trend.rename({'allpoints_level_0':dim1, 'allpoints_level_1':dim2})
    return da_ac

In [ ]:
SST_yrly_had[:,100,150].plot()
print(autocorrelation(SST_yrly_had[:,100,150]))

In [ ]:
%%time
# latlon       = (depth_lat_lon_names('ocn')[1:])
# SST_map_ctrl = xr_autocorrelation_2D(SST_yrly_ctrl, latlon)
# SST_map_ctrl.to_netcdf(f'{path_samoc}/SST/SST_autocorrelation_ctrl.nc')
# SST_map_rcp  = xr_autocorrelation_2D(SST_yrly_rcp , latlon)
# SST_map_rcp .to_netcdf(f'{path_samoc}/SST/SST_autocorrelation_rcp.nc' )

# latlon       = (depth_lat_lon_names('ocn_low')[1:])
# SST_map_lpd  = xr_autocorrelation_2D(SST_yrly_lpd , latlon)
# SST_map_lpd .to_netcdf(f'{path_samoc}/SST/SST_autocorrelation_lpd.nc' )
# SST_map_lpi  = xr_autocorrelation_2D(SST_yrly_lpi , latlon)
# SST_map_lpi .to_netcdf(f'{path_samoc}/SST/SST_autocorrelation_lpi.nc' )

latlon       = (depth_lat_lon_names('ocn_had')[1:])
SST_map_had  = xr_autocorrelation_2D(SST_yrly_had , latlon)
SST_map_had .to_netcdf(f'{path_samoc}/SST/SST_autocorrelation_had.nc' )

In [ ]:
SST_map_had

In [ ]:
np.all(SST_map_had.values==np.nan)

In [ ]:
SST_map_had[100:130, 100:130].values

In [ ]:
SST_map_had.plot()

In [ ]:
def regr_map_standard(run, index):
    assert run in ['ctrl','rcp','lpd', 'lpi']
    assert index in ['SOM', 'AMO', 'TPI']
    SST_regr_standard(index)
    ds = xr.open_dataset(f'{path_results}/SST/{index}_regr_{run}.nc', decode_times=False)
    regr_map(ds=ds, index=index, run=run)
    return


def regr_map_diff_times(run, index):
    assert run in ['lpd', 'lpi']
    assert index in ['SOM','AMO', 'TPI']
    if run=='lpd':
        times = ['200_1', '200_2', '412']
        SST_regr_lpd(index)
    elif run=='lpi':
        times = ['800_1', '800_2', '1480']
        SST_regr_lpi(index)
    for i in range(3):
        ds = xr.open_dataset(f'{path_results}/SST/{index}_regr_{run}_{times[i]}.nc', decode_times=False)
        regr_map(ds=ds, index=index, run=run, fn=times[i])
    return

In [ ]:
for index in ['SOM']:#, 'AMO', 'TPI']:
    print(index)
    for run in ['ctrl', 'rcp', 'lpd', 'lpi']:
        print(run)
        regr_map_standard(run, index)
        if run in ['lpd', 'lpi']:
            regr_map_diff_times(run, index)

In [ ]:
for index in ['SOM']:#, 'AMO', 'TPI']:
    print(index)
    for run in ['lpi']:#['ctrl', 'rcp', 'lpd', 'lpi']:
        print(run)
        regr_map_standard(run, index)
        if run in ['lpd', 'lpi']:
            regr_map_diff_times(run, index)

In [ ]:
for index in ['SOM']:#'AMO', 'TPI']:
    print(index)
    for run in ['lpd']:#['ctrl', 'rcp', 'lpd', 'lpi']:
        print(run)
        regr_map_standard(run, index)
        if run in ['lpd', 'lpi']:
            regr_map_diff_times(run, index)